In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from LSTM import PoseDataset, LSTM
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch

In [2]:
data = np.load('data.npy', allow_pickle=True)

X_train, X_test, y_train, y_test = train_test_split(data[:, 0], data[:, 1], test_size=0.2, random_state=42)

In [3]:
print('Training size: ', len(X_train), 'Testing size: ', len(X_test))

Training size:  1584 Testing size:  396


In [4]:
SEQUENCE_LENGTH = X_train[0].shape[0]
INPUT_SIZE = X_train[0].shape[1]
HIDDEN_SIZE = 128
NUM_LAYERS = 2
NUM_CLASSES = len(np.unique(np.concatenate((y_train, y_test))))
BATCH_SIZE = 32
NUM_EPOCHS = 100
LEARNING_RATE = 0.001
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [5]:
train_dataset = PoseDataset(X_train, y_train, SEQUENCE_LENGTH)
test_dataset = PoseDataset(X_test, y_test, SEQUENCE_LENGTH)

train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [28]:
X_train[1000][0].shape


(132,)

In [7]:
model = LSTM(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, NUM_CLASSES).to(DEVICE)

# Loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [8]:
# Train network
for epoch in tqdm(range(NUM_EPOCHS)):
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(DEVICE).float()
        targets = targets.to(DEVICE)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):

    if loader.dataset.train:
        print('Checking accuracy on training data')
    else:
        print('Checking accuracy on test data')

    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(DEVICE).float()
            y = y.to(DEVICE)
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(f'Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}')

    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

# Save the model checkpoint
torch.save(model.state_dict(), 'lstm.ckpt')



  0%|          | 0/100 [00:00<?, ?it/s]


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found object